In [145]:
import torch
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from torch.utils.data import dataset
from torch import Tensor,nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

def try_gpu(i=0):
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [146]:
device = try_gpu(0)

In [147]:
with open('../data/simple.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [148]:
train_iter = text.split('\n')[:5500]
test_iter = text.split('\n')[5500:]

In [149]:
print(len(train_iter),len(test_iter))

5500 867


In [150]:
tokenizer = get_tokenizer(None)
vocab = build_vocab_from_iterator(
    map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [151]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long)
            for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [152]:
train_data = data_process(train_iter)
test_data = data_process(test_iter)

In [153]:
train_data

tensor([21361,    51,    44,  ...,    93,    60,     4])

In [154]:
def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.
    Args:
        data: Tensor, shape [N]
        bsz: int, batch size
    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

In [117]:
# batch_size = 35
# eval_batch_size = 10

# train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
# test_data = batchify(test_data, eval_batch_size)

In [118]:
text = ['आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य']
#text = ['जनसंख्या']
sample_data = data_process(
    text)

In [119]:
sample_data

tensor([ 9979,     0,   483,   859,   281,     0,   859,   533,  3799,  1844,
        18893,   636,     0,   533,  3799])

In [120]:
sample_data = batchify(sample_data, 2)
print("Given word:", text[0])
sample_data

Given word: आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य


tensor([[ 9979,   533],
        [    0,  3799],
        [  483,  1844],
        [  859, 18893],
        [  281,   636],
        [    0,     0],
        [  859,   533]], device='cuda:0')

In [155]:
bptt = 35
train_data = batchify(train_data, bptt)  # shape [seq_len, batch_size]
test_data = batchify(test_data, bptt)


import math
def get_batch(source: Tensor, i: int) -> tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int
    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    #target = source[i+1:i+1+seq_len]
    return data, target

In [175]:





class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)
                             * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)





class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(
            d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]
        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [176]:
ntokens = len(vocab)  # size of vocabulary
emsize = 300  # embedding dimension
d_hid = 400  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4  # number of heads in nn.MultiheadAttention
dropout = 0.1  # dropout probability
model = TransformerModel(ntokens, emsize,nhead, d_hid,
                         nlayers, dropout).to(device)

In [177]:
criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [178]:
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [191]:
def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        #print(type(output))
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

In [192]:
softmax = nn.Softmax(dim=2)
softmax = nn.LogSoftmax(dim=2)

In [207]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_softmax = softmax(output)
            # output_softmax = softmax(output.view(-1, batch_size, ntokens))
            # output_softmax_permuted = output_softmax.permute(1, 0, 2)
            output_softmax_permuted = output_softmax.permute(1, 0, 2)
            indices = torch.argmax(output_softmax_permuted, dim=2)
            target_indices = targets.t()
            # print(output)
            output_flat = output.view(-1, ntokens)
            # total_loss += batch_size * criterion(output_flat, targets).item()
#            print('data')
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in data.t()]))
#             print(indices)
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in indices]))
#             print(len(targets))
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in target_indices]))
    return total_loss / (len(eval_data) - 1)
    return 0.2

In [235]:
import time
def generator(model: nn.Module, gen_data: Tensor):
    model.eval()
    temp_text = text
    for i in range(10):
        print('i:', i)
        src_mask = generate_square_subsequent_mask(bptt).to(device)
        batch_size = gen_data.size(0)
        if batch_size != bptt:
            src_mask = src_mask[:batch_size, :batch_size]
        output_softmax = model(gen_data, src_mask)
        output_softmax_permuted = output_softmax.permute(1, 0, 2)
        indices = torch.argmax(output_softmax_permuted, dim=2)
        print('next word: ', [vocab.lookup_tokens(list(index))
                              for index in indices][0][i])
        # target_indices = targets.t()
        # print(indices.shape)
        # print(indices)
        # temp_gen_data = [vocab.lookup_tokens(
        #    list(index)) for index in indices][0][i]
        # gen_data = [vocab.lookup_tokens(
        #    list(index)) for index in indices][0]
        # print(temp_text)
        # print([[vocab.lookup_tokens(list(index))
        #        for index in indices][0][i]])
#         temp_text = [[vocab.lookup_tokens(list(index))]
#                       for index in indices[0][i]]
        # print(temp_text)
#         temp_text = [' '.join(temp_text)]
        # print(temp_text)
        # gen_data = vocab.lookup_tokens((list(gen_data))) + list(temp_gen_data)
        # gen_data = torch.tensor(
        #    (list(gen_data[0])+list(indices[0][i]))).unsqueeze(0)
        # gen_data = torch.concat([gen_data, torch.tensor(indices[0][i])], dim=1)
        #gen_data = ' '.join(gen_data)
        #return temp_txt
#         gen_data = data_process(temp_text)
#         gen_data = batchify(gen_data, 1)
        #return gen_data


In [223]:
# Loop over epochs. Save the model if the validation loss is the best
# we've seen so far. Adjust the learning rate after each epoch.
import copy
best_val_loss = float('inf')
epochs = 4
best_model = None
print(len(vocab.get_stoi()))
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    eval_loss = evaluate(model, test_data)
    eval_ppl = math.exp(eval_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
         f'valid loss {eval_loss:5.2f} | valid ppl {eval_ppl:8.2f}')
    print('-' * 89)
    #best_model = copy.deepcopy(model)
    if eval_loss < best_val_loss:
       best_val_loss = eval_loss
       best_model = copy.deepcopy(model)

#     scheduler.step()
#save model
torch.save(best_model.state_dict(),'best_model.pt')
# Load saved model
# model.load_state_dict(torch.load(saved_path))
# model.to(device)

21362
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  7.75s | valid loss  0.00 | valid ppl     1.00
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time:  7.08s | valid loss  0.00 | valid ppl     1.00
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time:  7.10s | valid loss  0.00 | valid ppl     1.00
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   4 | time:  7.13s | valid loss  0.00 | valid ppl     1.00
--------------------------------------------------------------------

In [236]:
generator(best_model, sample_data)

i: 0
next word:  वर्षसम्ममा
i: 1
next word:  कार्यक्रम
i: 2
next word:  गरेर
i: 3
next word:  आर्थिक
i: 4
next word:  क्षेत्रको
i: 5
next word:  आर्थिक
i: 6
next word:  आर्थिक
i: 7


IndexError: list index out of range